In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d shubhamgoel27/dermnet

Dataset URL: https://www.kaggle.com/datasets/shubhamgoel27/dermnet
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
 99% 1.71G/1.72G [00:09<00:00, 230MB/s]
100% 1.72G/1.72G [00:09<00:00, 192MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dermnet.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ConvNeXtTiny
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# === CONFIG ===
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 7
NUM_CLASSES = 23  # Adjust if needed

# === LOAD DATA ===
train_ds = image_dataset_from_directory(
    "/content/train",
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

val_ds = image_dataset_from_directory(
    "/content/test",
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

# === DATA AUGMENTATION ===
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

# === PREFETCH ===
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

# === BASE MODEL ===
base_model = ConvNeXtTiny(
    include_top=False,
    input_shape=(224, 224, 3),
    weights="imagenet",
    pooling=None
)
base_model.trainable = True  # Fine-tune

# === MODEL BUILD ===
inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = base_model(x, training=True)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = tf.keras.Model(inputs, outputs)

# === COMPILE ===
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# === CALLBACKS ===
early_stop = EarlyStopping(patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_convnext_model.keras", save_best_only=True)

# === TRAIN ===
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[early_stop, checkpoint]
)


Found 15557 files belonging to 23 classes.
Found 4002 files belonging to 23 classes.
Epoch 1/7
487/487 ━━━━━━━━━━━━━━━━━━━━ 751s 1s/step - accuracy: 0.1755 - loss: 3.4419 - val_accuracy: 0.3488 - val_loss: 2.3510
Epoch 2/7
487/487 ━━━━━━━━━━━━━━━━━━━━ 693s 1s/step - accuracy: 0.3614 - loss: 2.3339 - val_accuracy: 0.4133 - val_loss: 2.0514
Epoch 3/7
487/487 ━━━━━━━━━━━━━━━━━━━━ 682s 1s/step - accuracy: 0.4664 - loss: 1.8634 - val_accuracy: 0.4635 - val_loss: 1.9026
Epoch 4/7
487/487 ━━━━━━━━━━━━━━━━━━━━ 668s 1s/step - accuracy: 0.5511 - loss: 1.5196 - val_accuracy: 0.5217 - val_loss: 1.6829
Epoch 5/7
487/487 ━━━━━━━━━━━━━━━━━━━━ 689s 1s/step - accuracy: 0.6458 - loss: 1.1968 - val_accuracy: 0.5537 - val_loss: 1.5976
Epoch 6/7
487/487 ━━━━━━━━━━━━━━━━━━━━ 669s 1s/step - accuracy: 0.7076 - loss: 0.9634 - val_accuracy: 0.5572 - val_loss: 1.6731
Epoch 7/7
487/487 ━━━━━━━━━━━━━━━━━━━━ 688s 1s/step - accuracy: 0.7677 - loss: 0.7460 - val_accuracy: 0.5920 - val_loss: 1.5697


In [ ]:
model.save("convnext_tiny_trained.h5")